In [ ]:
# export
import torch
import torch_geometric
from torch_geometric.data import HeteroData
from torch_geometric.datasets import OGB_MAG
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn import SAGEConv, to_hetero
import hiddenlayer as hl
from torchsummary import summary



In [ ]:
#export
transform = torch_geometric.transforms.ToUndirected()  # Add reverse edge types.
dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=transform)
data = dataset[0]

train_loader = torch_geometric.loader.NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 1,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=1,
    input_nodes=('paper', data['paper'].train_mask),
)
batch = next(iter(train_loader))

In [ ]:
for key, val in batch.x_dict.items():
    print(f"type input name:{key}, shape of tensor : {list(val.size())}")
for key, val in batch.edge_index_dict.items():
    print(f"type input name:{key}, shape of tensor : {list(val.size())}")

In [ ]:
#export
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels,
                 ):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)  #.jittable()
        self.conv2 = SAGEConv((-1, -1), out_channels)  #.jittable()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, batch.metadata(), aggr='sum')
with torch.no_grad():  # Initialize lazy modules.
    out = model(batch.x_dict, batch.edge_index_dict)
a = 1

In [ ]:
#export
def save_model_simple(py_model: torch.nn.Module):
    torch.save(py_model.state_dict(), "shmira_sd.h5")
    torch.save(py_model.state_dict(), "shmira_sd.pt")
    torch.save(py_model, "shmira.h5")
    torch.save(py_model, "shmira.pt")


a = 1

In [ ]:
# model_ts = torch.jit.script(model)
# with torch.no_grad():  # Initialize lazy modules.
#     out = model_ts(batch.x_dict, batch.edge_index_dict)
# torch.save(model_ts.state_dict(),"shmira_ts_sd.h5")
# torch.save(model_ts.state_dict(),"shmira_ts_sd.pt")
# torch.save(model_ts,"shmira_ts.h5")
# torch.save(model_ts,"shmira_ts.pt")

In [1]:
from nbdev.export import notebook2script

notebook2script()

OSError: [Errno 30] Read-only file system: '/settings.ini'